In [2]:
import sqlite3
con=sqlite3.connect('product2.db')
cur=con.cursor()


cur.execute('''CREATE TABLE IF NOT EXISTS products (date text,trans text,product_id int, product_name text,qty real,price float)''')

In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import findspark
findspark.init()
import os
spark=SparkSession.builder.appName('SparkSQL_UseCase').master('local[2]').getOrCreate()

In [4]:
cur.execute("INSERT INTO products VALUES('2023-04-05','credit card',1331,'Iphone',10,35.14)")

In [5]:
con.commit()
con.close()

In [9]:
spark=SparkSession.builder \
.master("local") \
.appName("SQLite JDBC") \
.config("spark.jars","C:\\Users\\pdharantej\\OneDrive - ALLEGIS GROUP\\Desktop\\TEK_Training\\5. Data Analysis\\db_&_jarfiles_spark_sql\\sqlite-jdbc-3.34.0.jar")\
.config("spark.driver.extraClassPath","C:\\Users\\pdharantej\\OneDrive - ALLEGIS GROUP\\Desktop\\TEK_Training\\5. Data Analysis\\db_&_jarfiles_spark_sql\\sqlite-jdbc-3.34.0.jar") \
.getOrCreate()

In [10]:
import os
os.getcwd()

'C:\\Users\\pdharantej\\OneDrive - ALLEGIS GROUP\\Desktop\\TEK_Training\\5. Data Analysis'

In [20]:
# products_df=spark.read.format("jdbc").option("url","jdbc:sqlite://{}/product2.db".format(os.getcwd())).option("driver","org.sqlite.JDBC").option("dbtable","products").load()

In [30]:
# userDetails = {'userName': 'root', 'pwd':'ThinkPad@66'}
# import pyspark
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.config("spark.jars", "C:\\Users\\pdharantej\\Downloads\\mysql-connector-java-8.0.32.jar") \
#     .master("local[*]").appName("PySpark_MySQL_test").getOrCreate()
# products_df = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/world") \
#     .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "country") \
#     .option("user", userDetails['userName']).option("password", userDetails['pwd']).load()

In [19]:
products_df.show()

NameError: name 'products_df' is not defined

In [ ]:
custDF=spark.read.format('csv').option('header','true').option('inferSchema','true').load('./Customers.csvs')
salesDF=spark.read.format('csv').option('header','true').option('inferSchema','true').load('./Sales.csv')


In [ ]:
custDF=custDF.withColumnRenamed('ID','customer_id')

In [ ]:
custDF.show(3)

In [ ]:
salesDF=salesDF.withColumnRenamed('Customer ID','customer_id')
salesDF.show(3)

In [ ]:
broadcastJoin=custDF.hint("broadcast").join(salesDF,"customer_id")
broadcastJoin.show()

In [ ]:
shuffleHashJoin = custDF.hint('shuffle_hash').join(salesDF, 'customer_id')
shuffleHashJoin.show(5)

In [ ]:
cartesianProduct = custDF.hint('shuffle_replicate_nl').join(salesDF)

In [ ]:
cartesianProduct.count()

In [ ]:
df1 = custDF.groupBy('company').count()
df1.rdd.getNumPartitions()

In [ ]:
spark.conf.set('spark.sql.adaptive.enabled', True)
spark.conf.set('spark.sql.adaptive.coalescePartitions.enabled', True)

In [ ]:
df1 = custDF.groupBy('company').count()
df1.rdd.getNumPartitions()

In [12]:
# Higher order functions

from pyspark.sql.functions import transform, when

In [14]:
df=spark.createDataFrame([(1, [1,2,3,4])], ('key', 'values'))
df.select(transform('values', lambda x: x*2).alias('doubled')).show()

+------------+
|     doubled|
+------------+
|[2, 4, 6, 8]|
+------------+



In [17]:
def alternate(x, i):
    return when(i%2==0, x).otherwise(-x)

In [18]:
df.select(transform('values', alternate).alias('alternated')).show()

+--------------+
|    alternated|
+--------------+
|[1, -2, 3, -4]|
+--------------+



In [28]:
spark.sql('select array(1,2,3) as orig_array, transform(array(1,2,3), x->x+1) as new').show()

+----------+---------+
|orig_array|      new|
+----------+---------+
| [1, 2, 3]|[2, 3, 4]|
+----------+---------+



In [29]:
arrayData=[(3,[1,2,3]),(4,[5,6,7,8])]
df=spark.createDataFrame(data=arrayData ,schema=['col1','col2'])
df.printSchema()
df.createOrReplaceTempView("mytesttable")
spark.sql("select * from mytesttable").show()

root
 |-- col1: long (nullable = true)
 |-- col2: array (nullable = true)
 |    |-- element: long (containsNull = true)

+----+------------+
|col1|        col2|
+----+------------+
|   3|   [1, 2, 3]|
|   4|[5, 6, 7, 8]|
+----+------------+



In [31]:
spark.sql('select col1, col2 as origin_array, transform(filter(col2, (x, i)->i=2), x->x+5) as new_array from mytesttable').show(truncate=False)

+----+------------+---------+
|col1|origin_array|new_array|
+----+------------+---------+
|3   |[1, 2, 3]   |[8]      |
|4   |[5, 6, 7, 8]|[12]     |
+----+------------+---------+



In [32]:
spark.sql('select col1, col2 as origin_array, transform(col2, (x, i)->x+col1) as new_array from mytesttable').show(truncate=False)

+----+------------+---------------+
|col1|origin_array|new_array      |
+----+------------+---------------+
|3   |[1, 2, 3]   |[4, 5, 6]      |
|4   |[5, 6, 7, 8]|[9, 10, 11, 12]|
+----+------------+---------------+



In [33]:
from pyspark.sql.functions import exists

In [34]:
df=spark.createDataFrame([(1, [1,2,3,4]), (2, [3,-1,0])], ('key', 'values'))
df.select(exists('values', lambda x: x<0).alias('any_negative')).show()

+------------+
|any_negative|
+------------+
|       false|
|        true|
+------------+



In [36]:
from pyspark.sql.functions import forall

In [37]:
df=spark.createDataFrame(
    [(1, ['bar']), (2,['foo', 'bar']), (3, ['foobar', 'foo'])],
    ('key', 'values')
)
df.select(forall('values', lambda x: x.rlike('foo')).alias('all_foo')).show()

+-------+
|all_foo|
+-------+
|  false|
|  false|
|   true|
+-------+

